In [20]:
import numpy as np 
import numpy.typing as npt 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import re
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.decomposition import PCA

from scipy.spatial import distance
from scipy.stats import mode

from dollarpy import Recognizer, Template, Point

from math import sqrt
from typing import TextIO

In [2]:
USER_IDS = range(0, 10)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


### Load datasets

In [7]:
def load_datasets(domain: str):
	dir_path = f"datasets/{domain}"

	X = []
	Y = []

	# matrix (number_drawn, user_id)
	# at each cell we have a list of multiple try of a specific number drawing (number_drawn)
	# drawn by a specific user (user_id), each try contains a list of data points (x, y, z)
	hand_gesture_data_matrix = np.zeros((10, 10), dtype=object)
	for i in range(0, 10):
		for j in range(0, 10):
			hand_gesture_data_matrix[i, j] = []

	for filename in tqdm(range(1, 1001)):
		file_path = f"{dir_path}/{filename}.txt"
		with open(file=file_path, mode="r") as f:
			# get the target, user_id 
			# and a list of positions vectors \in \R^3: <x, y, z>
			# that represents the drawing
			number_drawn, user_id, gesture_datas = load_gesture_data(file=f)

			X.append(gesture_datas)
			Y.append(number_drawn)

			hand_gesture_data_matrix[number_drawn - 1, user_id - 1].append(gesture_datas)

	return np.array(X), np.array(Y), hand_gesture_data_matrix

def load_gesture_data(file: TextIO):
	"""Load a hand gesture data file structured in the following way:
	-----
	Domain id = <domain-id>
	Class id = <class-id>
	User id = <user-id>

	"<x>,<y>,<z>,<t>"
	...
	-----

	Parameters
	----------
	file : TextIOWrapper
		Hand gesture data file

	Returns
	-------
	int
		Number drawn
	int 
		User id
	npt.NDArray, shape (n_timepoints, vec_dimension)
		Time series of hand gesture data points
	"""
	lines = file.readlines()
	
	number_drawn = lines[1].strip()
	match_number = re.search("=", number_drawn)
	number_drawn = int(match_number.string[match_number.end():].strip())

	user_id = lines[2].strip()
	match_user = re.search("=", user_id)
	user_id = int(match_user.string[match_user.end():].strip())

	gesture_datas = []

	for row in range(5, len(lines)):
		gesture_data = lines[row].split(",")
		# we only keep <x, y, z> coordinates
		gesture_data = np.array([float(data.strip()) for data in gesture_data[0:-1]])
		gesture_datas.append(gesture_data)
	
	return number_drawn, user_id, gesture_datas

X, y, hand_gesture_data_matrix = load_datasets(domain="Domain01")

  0%|          | 0/1000 [00:00<?, ?it/s]

/var/folders/zn/_2h7w2j547357vpm3_5pxrsr0000gn/T/ipykernel_16890/2319885195.py:28: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X), np.array(Y), hand_gesture_data_matrix


### Cross-validation

In [ ]:
def split_dataset(X, Y, user_id):
	slices = range(user_id * 100, 100 + (user_id * 100))
		
	X_train = np.delete(X, slices)
	y_train = np.delete(Y, slices)

	X_test = X[slices]
	y_test = Y[slices]

	return X_train, X_test, y_train, y_test

def user_independent_cross_validation(estimator, X, y):
	y_preds = []
	accuracies = []

	for user_id in USER_IDS:
		X_train, X_test, y_train, y_test = split_dataset(X, y, user_id=user_id)

		estimator.fit(X_train, y_train)
		y_pred = estimator.predict(X_test)

		print(y_test)
		print(y_pred)

		accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
		accuracies.append(accuracy)

		y_preds.append(y_pred)

	accuracy_mean = np.mean(accuracies)
	accuracy_std = np.std(accuracies)
	
	return accuracy_mean, accuracy_std

def user_dependent_cross_validation(knn):
	pass

### Preprocessing

In [8]:
# standardize data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

### Dynamic Time Warping

In [9]:
# https://towardsdatascience.com/dynamic-time-warping-3933f25fcdd
def dtw_distance(ts_a: npt.NDArray, ts_b: npt.NDArray, window: int = 3) -> float:
	"""Compute the DTW distance for two time series

	Parameters
	----------
	ts_a : npt.NDArray, shape (n_timepoints, vec_dimension)
		Time series of hand gesture data points
	ts_b : npt.NDArray, shape (m_timepoints, vec_dimension)
		Time series of hand gesture data points
	window : int, optional
		Window size, by default 3

	Returns
	-------
	float
		_description_
	"""
	n = len(ts_a)
	m = len(ts_b)

	DTW_matrix = np.full((n+1, m+1), fill_value=np.inf)
	DTW_matrix[0, 0] = 0.0

	window = max(window, abs(n - m))

	for i in range(0, n+1):
		for j in range(max(0, i - window), min(m, i + window) + 1):
			DTW_matrix[i, j] = 0.0

	for i in range(1, n+1):
		for j in range(max(1, i - window), min(m, i + window) + 1):
			# manhattan distance
			cost = distance.sqeuclidean(ts_a[i-1], ts_b[j-1])
			optimal_warping_path = min(
				DTW_matrix[i-1, j], # insertion
				DTW_matrix[i, j-1], # deletion
				DTW_matrix[i-1, j-1] # match
			)

			DTW_matrix[i, j] = cost + optimal_warping_path
	
	return sqrt(DTW_matrix[n, m])

In [10]:
# CAVEAT: ONLY WORKS FOR MONODIMENSIONAL TIME SERIES...
def lb_keogh(ts_query: npt.NDArray, ts_candidate: npt.NDArray, window: int) -> float:
	"""Implement LB Keogh algorithm which is a fast lowerbounding method for constrained DTW distance

	Parameters
	----------
	ts_query : npt.NDArray, shape (n_timepoints, vec_dimension)
		Query time series to compare with the envelope of the candidate time series
	ts_candidate : npt.NDArray, shape (m_timepoints, vec_dimension)
		Candidate time series, used to compute the envelope.
	window : int
		Windows size to use for the envelope generation

	Returns
	-------
	float
		Distance between the query time series and the envelope of the candidate time series
	"""
	n = min(len(ts_query), len(ts_candidate))
	lb_tot = 0

	for i in range(0, n):
		start = (i - window if i - window >= 0 else 0)
		stop = (i + window if i + window <= n - 1 else n - 1)

		lower_bound = np.amin(ts_candidate[start:stop + 1], axis=0)
		upper_bound = np.amax(ts_candidate[start:stop + 1], axis=0)

		delta = []

		# for each coordinate of the position vector 
		for j in range(0, 3):
			if t[i][j] > upper_bound[j]:
				delta.append(t[i][j] - upper_bound)
			elif t[i][j] < lower_bound[j]:
				delta.append(lower_bound - t[i][j])
			
		lb_tot += np.linalg.norm(delta)

	return lb_tot

In [11]:
# https://nbviewer.org/github/markdregan/K-Nearest-Neighbors-with-Dynamic-Time-Warping/blob/master/K_Nearest_Neighbor_Dynamic_Time_Warping.ipynb

# http://alexminnaar.com/2014/04/16/Time-Series-Classification-and-Clustering-with-Python.html

class KNN:
	def __init__(self, n_neighbors = 1, window = 3, distance_fun = dtw_distance):
		self.n_neigbors = n_neighbors
		self.distance_fun = distance_fun
		self.window = window

		self.X = None 
		self.y = None
	
	def fit(self, X: npt.NDArray, y: npt.NDArray):
		"""Fit the model using X as training data and y as target

		Parameters
		----------
		X : npt.NDArray, shape (n_samples, n_timepoints, vec_dimension)
			Training data
		y : npt.NDArray, shape (n_samples,)
			Target data
		"""
		self.X = X
		self.y = y

	def distance_matrix(self, X_train: npt.NDArray, X_test: npt.NDArray):
		n = len(X_test)
		m = len(X_train)
		distance_matrix = np.zeros((n, m))

		for i in tqdm(range(0, n)):
			min_dist = np.inf
			for j in range(0, m):
				#if lb_keogh(X_train[j], X_test[i], window=self.window) < min_dist:
				dist = self.distance_fun(X_test[i], X_train[j], window=self.window)
				distance_matrix[i, j] = dist
					#if dist < min_dist:
						#min_dist = dist
		
		return distance_matrix

	def predict(self, X: npt.NDArray) -> npt.NDArray:
		"""Predict the target values (here: class label) for the given testing dataset

		Parameters
		----------
		X : npt.NDArray, shape (n_samples, n_timepoints, vec_dimension)
			Testing data

		Returns
		-------
		npt.NDArray, shape (n_samples,)
			Array of predicted target values
		"""
		distance_matrix = self.distance_matrix(self.X, X)

		knn_idx = distance_matrix.argsort()[:, :self.n_neigbors]
		knn_labels = self.y[knn_idx]

		prediction = mode(knn_labels, axis=1)[0]

		return prediction.ravel()

In [13]:
def plot_confusion_matrix(y_true, y_pred):
	conf_matrix = np.array(confusion_matrix(y_true=y_true, y_pred=y_pred))
	sns.heatmap(conf_matrix, annot=True)
	plt.show()

In [82]:
knn = KNN(n_neighbors=1, window=3, distance_fun=DTW_distance)

dtw_accuracy_mean, dtw_accuracy_std = user_independent_cross_validation(estimator=knn, X=X, y=y)

print("DOMAIN 01 - DYNAMIC TIME WARPING")

print("USER INDEPENDENT CROSS VALIDATION")
print(f"mean accuracy: {np.round(dtw_accuracy_mean, 3)}")
print(f"std accuracy: {np.round(dtw_accuracy_std, 3)}")

  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[ 1  1 10  1  8  8  8  8  8  8  2  2  2  3  2  2  5  5  5  5  3  3  3  3
  3  3  3  3  3  3  6  6  4  4  4  4  4  6  6  7  7  5  8  4  8  8  8  8
  8  8  8  8  8  8 10  5 10 10 10 10  7 10  8 10 10  8  8  8  7  8  8  6
 10  5  5 10  8  8  8  8  8  8  8  8  8  8  6  6  6  8 10 10 10 10 10 10
 10 10 10 10]


/var/folders/zn/_2h7w2j547357vpm3_5pxrsr0000gn/T/ipykernel_1181/2559922313.py:61: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  prediction = mode(knn_labels, axis=1)[0]


  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[ 7  7  9  7  8  1  1  7  9  1  8  3  2  3  2  3  2  2  2  3  8  3  3  3
  3  3  3  3  3  2  8  4 10  8  8  6  6  6  6  6  5  4  5  5  5  5  5  5
  5  5  4  4  4  4  4  4  4  6  4  7  7  7  7  7  7  7  7  7  7  7  5 10
 10 10 10 10  5 10  5  8  6  8  3  3  3  3  3  3  3  8  6 10  5 10 10 10
 10 10 10  8]


...

  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[8 8 8 8 8 8 8 8 8 8 2 2 2 2 2 7 7 7 4 7 7 3 3 2 3 3 2 2 3 3 5 4 4 4 4 5 4
 6 1 4 4 5 4 4 5 5 5 5 5 4 7 4 4 6 4 1 4 4 6 4 8 8 8 7 8 7 7 7 7 7 8 8 8 8
 8 8 8 8 4 8 1 1 8 1 1 1 1 9 9 9 8 8 8 8 8 8 8 8 8 8]


/var/folders/zn/_2h7w2j547357vpm3_5pxrsr0000gn/T/ipykernel_1181/2559922313.py:61: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  prediction = mode(knn_labels, axis=1)[0]


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

### $P Recognizer

In [35]:
class PointCloudRecognizer:
	def __init__(self):
		self.recognizer = None

	def fit(self, X: npt.NDArray, y: npt.NDArray):
		"""_summary_

		Parameters
		----------
		X : npt.NDArray, shape (n_samples, n_timepoints, vec_dimension)
			Training data
		y : npt.NDArray, shape (n_samples,)
			Training target values

		Returns
		-------
		_type_
			_description_
		"""
		self.X = X
		self.y = y

		training_data = []
		# for each vector of data points
		for (gesture, label) in zip(X, y):
			# we have 3D gesture data points 
			# $p-recognizer only accept 2D data points so we need to reduce to 2 dimensions
			pca = PCA(n_components=2)
			gesture_2d = pca.fit_transform(gesture)

			# convert each data points to point object
			points = []
			for (x1, x2) in gesture_2d:
				point = Point(x1, x2)
				points.append(point)
			
			training_data.append(Template(label, points))
		
		self.recognizer = Recognizer(training_data)
		return self.recognizer
			
	def predict(self, X: npt.NDArray):
		"""_summary_

		Parameters
		----------
		X : npt.NDArray, shape (n_samples, n_timepoints, vec_dimension)
			Testing data

		Returns
		-------
		_type_
			_description_
		"""
		if not self.recognizer:
			raise ValueError("You must fit the model first.") 

		results = []

		for gesture in tqdm(X):
			pca = PCA(n_components=2)
			gesture_2d = pca.fit_transform(gesture)

			points = []
			for (x1, x2) in gesture_2d:
				point = Point(x1, x2)
				points.append(point)

			result = self.recognizer.recognize(points)
			results.append(result[0])

		return results

recognizer = PointCloudRecognizer()
pcr_accuracy_mean, pcr_accuracy_std = user_independent_cross_validation(estimator=recognizer, X=X, y=y)

print("DOMAIN 01 - $P Recognizer")

print("USER INDEPENDENT CROSS VALIDATION")
print(f"mean accuracy: {np.round(pcr_accuracy_mean, 3)}")
print(f"std accuracy: {np.round(pcr_accuracy_std, 3)}")

  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 8, 2, 2, 2, 3, 3, 3, 3, 7, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 7, 4, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 7, 6, 6, 10, 8, 6, 7, 6, 7, 7, 7, 7, 7, 7, 1, 1, 1, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 7, 10, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 7, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 4, 8, 8, 8, 10, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


  0%|          | 0/100 [00:00<?, ?it/s]

...

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 7, 3, 3, 7, 3, 3, 3, 3, 3, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 2, 5, 5, 5, 5, 5, 1, 1, 6, 6, 1, 6, 6, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 5, 2, 2, 3, 2, 6, 7, 2, 6, 3, 3, 3, 3, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 7, 7, 7, 7, 1, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 5, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


  0%|          | 0/100 [00:00<?, ?it/s]

[ 1  1  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  2  2  2  3  3  3  3
  3  3  3  3  3  3  4  4  4  4  4  4  4  4  4  4  5  5  5  5  5  5  5  5
  5  5  6  6  6  6  6  6  6  6  6  6  7  7  7  7  7  7  7  7  7  7  8  8
  8  8  8  8  8  8  8  8  9  9  9  9  9  9  9  9  9  9 10 10 10 10 10 10
 10 10 10 10]
[7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 9, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 10, 7, 9, 7, 7, 9, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 6, 10, 10]
DOMAIN 01 - $P Recognizer
USER INDEPENDENT CROSS VALIDATION
mean accuracy: 0.925
std accuracy: 0.029


### Neural Network